In [37]:
import pandas as pd
songs = pd.read_csv("./hackathon_song_normalize.csv", encoding = "ISO-8859-1")
artists_infos = pd.read_csv("./hackathon_artists.csv", encoding = "ISO-8859-1")

In [38]:
df = songs.copy()
df['genre'] = df['genre'].str.split(',')
# Then you can explode the lists into separate rows
df = df.explode('genre')
df['genre'] = df['genre'].apply(lambda x : x.lower().replace(" ",""))
df = df.groupby('ID Artist')['genre'].apply(set).reset_index()
df["genre"] = df["genre"].apply(lambda x : list(x))
df['genre'] = df['genre'].apply(lambda x : ', '.join(x))

In [41]:
songs[songs["artist"]=='Alanis Morissette']

,artist,ID Artist,name,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre,Column1
285,Alanis Morissette,27,Hands Clean,269400,False,2002,57,0.513,0.82,7,-5.428,1,0.0299,0.00192,0.000003,0.504,0.52,99.952,"pop, Folk/Acoustic",NaN


In [23]:
# Artist aggregate
aggregate_df = songs.groupby('artist').agg({
    'duration_ms': 'mean',
    'explicit': 'mean',
    'popularity': 'mean',
    'danceability': 'mean',
    'energy': 'mean',
    'loudness': 'mean',
    'speechiness': 'mean',
    'acousticness': 'mean',
    'instrumentalness': 'mean',
    'liveness': 'mean',
    'tempo': 'mean',
    'valence': 'mean',
    'year': ['min', 'max'],
    'name': 'size'
})

# # Flatten the multi-level column names
def agg_col(col):
    if col[0] == 'year':
        return '_'.join(col).strip()
    else:
        return col[0]
aggregate_df.columns = [agg_col(col) for col in aggregate_df.columns.values]

# # If you want to keep 'artist' as a column and not an index
aggregate_df.reset_index(level=0, inplace=True)
aggregate_df.to_csv("hackathon_song_normalize_artist_grouped.csv")
ids_artists = songs.drop_duplicates(subset='artist')[["ID Artist", "artist"]]
aggregate_df = aggregate_df.merge(ids_artists, how='left')
del artists_infos['artist']
aggregate_df.merge(artists_infos, how='left', on = 'ID Artist').merge(df, how='left', on='ID Artist').drop_duplicates(subset='artist').to_csv("artists_infos_musics.csv")

In [9]:
# genres aggregate
df = songs.copy()
df['genre'] = df['genre'].str.split(',')
# Then you can explode the lists into separate rows
df = songs.explode('genre')
df['genre'] = df['genre'].apply(lambda x : x.lower().replace(" ",""))
# Assuming df is your DataFrame
aggregate_df = df.groupby('genre').agg({
    'duration_ms': 'mean',
    'explicit': 'mean',
    'popularity': 'mean',
    'danceability': 'mean',
    'energy': 'mean',
    'loudness': 'mean',
    'speechiness': 'mean',
    'acousticness': 'mean',
    'instrumentalness': 'mean',
    'liveness': 'mean',
    'tempo': 'mean',
    'valence': 'mean',
    'year': ['min', 'max'],
    'name': 'size'
})

# # Flatten the multi-level column names
def agg_col(col):
    if col[0] == 'year':
        return '_'.join(col).strip()
    else:
        return col[0]
aggregate_df.columns = [agg_col(col) for col in aggregate_df.columns.values]

# # If you want to keep 'artist' as a column and not an index
aggregate_df.reset_index(level=0, inplace=True)
aggregate_df.to_csv("hackathon_song_normalize_genre_grouped.csv")


In [11]:
artists_infos

,artist,ID Artist,gender,age,type,country,city_1,district_1,city_2,district_2,city_3,district_3
0,Drake,267,male,33,person,CA,NaN,NaN,Toronto,NaN,NaN,NaN
1,Post Malone,733,male,25,person,US,NaN,NaN,Syracuse,NaN,NaN,NaN
2,Ed Sheeran,277,male,29,person,GB,NaN,NaN,Halifax,NaN,NaN,NaN
3,J Balvin,409,male,35,person,CO,NaN,NaN,Medellín,NaN,NaN,NaN
4,Bad Bunny,86,male,26,person,PR,NaN,NaN,San Juan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
625,Rizzle Kicks,763,mixed,12,group,GB,NaN,NaN,Brighton,NaN,NaN,NaN
626,Phil Collins,720,male,69,person,GB,NaN,NaN,NaN,Hounslow,NaN,NaN
627,Waka Flocka Flame,984,male,34,person,US,NaN,NaN,South Jamaica,NaN,NaN,NaN
628,Jessica Simpson,440,female,40,person,US,NaN,NaN,Abilene,NaN,NaN,NaN
